In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Building molecular Hamiltonians
===============================

::: {.meta}
:property=\"og:description\": Learn how to build electronic Hamiltonians
of molecules.

:property=\"og:image\":
<https://pennylane.ai/qml/_images/water_structure.png>
:::

::: {.related}
tutorial\_vqe A brief overview of VQE
:::

*Author: Alain Delgado --- Posted: 02 August 2020. Last updated: 25 June
2022.*

The ultimate goal of computational quantum chemistry is to unravel the
quantum effects that determine the structure and properties of
molecules. Reaching this goal is challenging since the characteristic
energies associated with these effects, e.g., the electronic correlation
energy, are typically a tiny fraction of the total energy of the
molecule.

Accurate molecular properties can be computed from the wave function
describing the interacting electrons in a molecule. The **electronic**
wave function $\Psi(r)$ satisfies the [Schrödinger
equation](https://en.wikipedia.org/wiki/Schr%C3%B6dinger_equation)

$$H_e \Psi(r) = E \Psi(r),$$

where $H_e$ and $E$ denote the electronic Hamiltonian and the total
energy of the molecule, respectively. When solving the latter equation,
the nuclei of the molecule can be treated as point particles whose
coordinates are fixed. In this approximation, both the total energy and
the electronic Hamiltonian depend parametrically on the nuclear
coordinates.

In this tutorial, you will learn how to use PennyLane to build a
representation of the electronic Hamiltonian $H_e$ that can be used to
perform **quantum** simulations of molecules. First, we show how to
define the structure of the molecule in terms of the symbols and the
coordinates of the atoms. Next, we describe how to solve the
[Hartree-Fock
equations](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)
for the target molecule. Finally, we discuss some advanced features that
can be used to simulate more complicated systems.

Let\'s get started!

Defining the molecular structure
--------------------------------

In this example we construct the electronic Hamiltonian of the water
molecule.

![](../demonstrations/quantum_chemistry/water_structure.png){.align-center
width="30.0%"}

The structure of a molecule is defined by the symbols and the nuclear
coordinates of its constituent atoms. It can be specified using
different [chemical file
formats](https://en.wikipedia.org/wiki/Chemical_file_format). Within
PennyLane, the molecular structure is defined by providing a list with
the atomic symbols and a one-dimensional array with the nuclear
coordinates in [atomic
units](https://en.wikipedia.org/wiki/Hartree_atomic_units).


In [ ]:
import numpy as np

symbols = ["H", "O", "H"]
coordinates = np.array([-0.0399, -0.0038, 0.0, 1.5780, 0.8540, 0.0, 2.7909, -0.5159, 0.0])

The `~.pennylane.qchem.read_structure`{.interpreted-text role="func"}
function can also be used to read the molecular geometry from an
external file.


In [ ]:
from pennylane import qchem

symbols, coordinates = qchem.read_structure("h2o.xyz")

The xyz format is supported.

Solving the Hartree-Fock equations
==================================

The molecule\'s electronic Hamiltonian is commonly represented using the
[second-quantization](https://en.wikipedia.org/wiki/Second_quantization)
formalism, which we will explore in more detail in the next section. To
that aim, a basis of **single-particle** states needs to be chosen. In
quantum chemistry these states are the [molecular
orbitals](https://en.wikipedia.org/wiki/Molecular_orbital) which
describe the wave function of a single electron in the molecule.

Molecular orbitals are typically represented as a linear combination of
**atomic orbitals**. The expansion coefficients in the atomic basis are
calculated using the [Hartree-Fock (HF)
method](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method). In
the HF approximation, each electron in the molecule is treated as an
**independent** particle that moves under the influence of the Coulomb
potential due to the nuclei, and a mean field generated by all other
electrons. The optimized coefficients are precisely what we need to
build the second-quantized Hamiltonian.

PennyLane provides a differentiable Hartree-Fock solver and the
functionality to construct a fully-differentiable molecular Hamiltonian.

Building the Hamiltonian
========================

In the second quantization formalism, the electronic wave function of
the molecule is represented in the occupation number basis. For $M$
*spin* molecular orbitals, the elements of this basis are labelled as
$\vert n_0, n_1, \dots, n_{M-1} \rangle$, where $n_i = 0,1$ indicates
the occupation of each orbital. In this representation, the electronic
Hamiltonian is given by

$$H = \sum_{p,q} h_{pq} c_p^\dagger c_q +
\frac{1}{2} \sum_{p,q,r,s} h_{pqrs} c_p^\dagger c_q^\dagger c_r c_s,$$

where $c^\dagger$ and $c$ are the electron creation and annihilation
operators, respectively, and the coefficients $h_{pq}$ and $h_{pqrs}$
denote the one- and two-electron Coulomb integrals evaluated using the
Hartree-Fock orbitals.

We can use the states of $M$ qubits to encode any element of the
occupation number basis

$$\vert n_0, n_1, \dots, n_{M-1} \rangle \rightarrow \vert q_0 q_1 \cdots q_{M-1} \rangle.$$

This implies that we need to map the fermionic operators onto operators
that act on qubits. This can be done by using the
[Jordan-Wigner](https://en.wikipedia.org/wiki/Jordan-Wigner_transformation)
transformation which allows us to decompose the fermionic Hamiltonian
into a linear combination of the tensor product of Pauli operators

$$H = \sum_j C_j \otimes_i \sigma_i^{(j)},$$

where $C_j$ is a scalar coefficient and $\sigma_i$ represents an element
of the Pauli group $\{ I, X, Y, Z \}$.

In PennyLane we have the
`~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text role="func"}
function which encapsulates all the steps explained above. It simplifies
the process of building the electronic Hamiltonian to a single line of
code. We just need to input the symbols and the nuclear coordinates of
the molecule, as shown below:


In [ ]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Advanced features
=================

The `~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text
role="func"} function allows us to define additional keyword arguments
to solve the Hartree-Fock equations of more complicated systems. The net
charge of the molecule may be specified to simulate positively or
negatively charged molecules. For a neutral system we choose


In [ ]:
charge = 0

We can also specify the [spin
multiplicity](https://en.wikipedia.org/wiki/Multiplicity_(chemistry)).
For the water molecule, which contains ten electrons, the [Slater
determinant](https://en.wikipedia.org/wiki/Slater_determinant) resulting
from occupying the five lowest-energy orbitals with two *paired*
electrons per orbital has spin multiplicity one. Alternatively, if we
define an occupation where the first four orbitals are doubly occupied
and the next two are singly occupied by *unpaired* electrons, the HF
state will have multiplicity three.

| 

![](../demonstrations/quantum_chemistry/hf_references.png){.align-center
width="50.0%"}

| 

For the neutral water molecule we have,


In [ ]:
multiplicity = 1

As mentioned above, molecular orbitals are represented as a linear
combination of atomic orbitals which are typically modeled as
[Gaussian-type
orbitals](https://en.wikipedia.org/wiki/Gaussian_orbital). We can
specify different types of [Gaussian atomic
bases](https://www.basissetexchange.org/). In this example we choose a
[minimal basis
set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)#Minimal_basis_sets).


In [ ]:
basis_set = "sto-3g"

PennyLane also allows us to define an active space to perform quantum
simulations with a reduced number of qubits. This is done by classifying
the molecular orbitals as core, active, and external orbitals:

-   Core orbitals are always occupied by two electrons.
-   Active orbitals can be occupied by zero, one, or two electrons.
-   The external orbitals are never occupied.

Within this approximation, a certain number of **active electrons** are
allowed to populate a finite set of **active orbitals**.

![](../demonstrations/quantum_chemistry/sketch_active_space.png){.align-center
width="40.0%"}

::: {.note}
::: {.title}
Note
:::

The number of active **spin-orbitals** determines the **number of
qubits** required to perform the quantum simulations.
:::

For the water molecule in a minimal basis set we have a total of ten
electrons and seven molecular orbitals. In this example we define an
symmetric active space with four electrons and four active orbitals
using the `~.pennylane.qchem.active_space`{.interpreted-text
role="func"} function:


In [ ]:
electrons = 10
orbitals = 7
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)

Viewing the results:


In [ ]:
print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

Finally, we use the
`~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text role="func"}
function to build the resulting Hamiltonian of the water molecule:


In [ ]:
H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")
print(H)

In this case, since we have truncated the basis of molecular orbitals,
the resulting observable is an approximation of the Hamiltonian
generated in the section `hamiltonian`{.interpreted-text role="ref"}.

OpenFermion-PySCF backend
=========================

The `~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text
role="func"} function can also be used to construct the molecular
Hamiltonian with a non-differentiable backend that uses the
[OpenFermion-PySCF](https://github.com/quantumlib/OpenFermion-PySCF)
plugin interfaced with the electronic structure package
[PySCF](https://github.com/sunqm/pyscf). This backend can be selected by
setting `method='pyscf'` in
`~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text
role="func"}:


In [ ]:
H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method="pyscf")
print(H)

This backend requires the `OpenFermion-PySCF` plugin to be installed by
the user with

``` {.bash}
pip install openfermionpyscf
```

Additionally, if you have built your electronic Hamiltonian
independently using
[OpenFermion](https://github.com/quantumlib/OpenFermion) tools, it can
be readily converted to a PennyLane observable using the
`~.pennylane.qchem.import_operator`{.interpreted-text role="func"}
function.

You have completed the tutorial! Now, select your favorite molecule and
build its electronic Hamiltonian. To see how simple it is to implement
the VQE algorithm to compute the ground-state energy of your molecule
using PennyLane, take a look at the tutorial
`tutorial_vqe`{.interpreted-text role="doc"}.

References
==========

About the author
================
